# Query/Context Dataset Generation
***

This notebook walks students through the process of generating datasets of query/context pairs which can be used for two primary purposes:
- Fine-tuning an embedding model
- Serve as ground truth for retrieval evaluation

In [7]:
from retrieval_evaluation import QueryContextGenerator
from llama_index.finetuning import EmbeddingQAFinetuneDataset
from prompt_templates import qa_generation_prompt
from preprocessing import FileIO
from rich import print
import pandas as pd
import os

from dotenv import load_dotenv
env = load_dotenv('.env', override=True)

In [15]:
#instantiate generate with openai_key, model_id default is 'gpt-3.5-turbo-0613'
generator = QueryContextGenerator(openai_key=os.environ['OPENAI_API_KEY'])

### Load raw data
Load raw data from parquet file.  Raw data should be in the same format as the dataset (corpus) created in [Notebook 1](https://github.com/americanthinker/vectorsearch-applications/blob/main/1-Data_Preprocessing_Week1_COLAB.ipynb). 

In [8]:
data_path = '../impact_theory_minilm_256.parquet'
data = FileIO().load_parquet(data_path)
len(data)

Shape of data: (26448, 12)
Memory Usage: 2.42+ MB


26448

### Data Length Analysis
Conduct an analysis of the length of the content chunks.  Can use either raw words or tokens to assess length.  The main point here is to get a sense of the mean length of content chunks in the data and to set the `total_chars` param in the `clean_validate_data` method with an appropriate value.

In [17]:
#in this example the mean content length is @ 1,000
lengths = [len(d['content']) for d in data]
df = pd.DataFrame(lengths)
df.describe() 

,0
count,26448.000000
mean,991.729053
std,126.344870
min,4.000000
25%,944.000000
50%,1005.000000
75%,1060.000000
max,1974.000000


### Split Data

The `train_val_split` function will clean and validate the raw data as a first step and then split into user defined train/val splits.  
- Cleaning simply strips the keys from the data that are not needed for the query/content generation process
- Validation consists of ensuring that only content chunks of length > `total_chars` are passed to the LLM (this step prevents the LLM from asking questions from sparse context)

Users define the number of training samples and validation samples to generate.  Number of questions per content chunk can also be set to more than 1, however a note of caution:
- Setting `num_questions_per_chunk` > 1 saves time (and money) by asking more than one question per content chunk, however, the dataset will be less diverse.  There is also the potential for the model to generate lower quality questions if the content chunk isn't large enough or meaningful enough to generate more than one question from the content.
- Retrieval evaluation results from fine-tuning an embedding model with 200-300 training samples showed an uptick of 5-10% points.  Upper bound on retrieval improvement as a funtion of training sample size is yet to be determined (have fun pushing the boundaries! 👊)
- A validation data set is not required for seeing improvement from fine tuning.  The addition of a validation dataset, however, allows a user to test the results of fine tuning on an unseen dataset. 

In [19]:
#split data into train/val sets
#in this example we are creating a training set of n=10, val set of n=5, and asking the LLM to only ask 1 question per chunk. 
train, val = generator.train_val_split(data, 10, 5, 1, total_chars=950)

Length Training Data: 10
Length Validation Data: 5


### Generate QA pairs

To generate query/context pairs we need to pass in our cleaned data splits, a question asking generation prompt, and the number of questions per chunk (needs to be same value passed into the `train_val_split` function.
The `qa_generation_prompt` is already preconfigured and supplies the LLM with additional context about the Impact Theory show to ensure high quality questions are asked given the additional context.   
Print out the prompt to see what is being asked of the model:

In [8]:
print(qa_generation_prompt)

Impact Theory episode summary and episode guest are below:

---------------------
Summary: {summary}
---------------------
Guest: {guest}
---------------------
Given the Summary and Guest of the episode as context use the following randomly selected transcript section \  
of the episode and not prior knowledge, generate questions that can be answered by the transcript section: 

---------------------
{transcript}
---------------------

Your task is to create {num_questions_per_chunk} questions that can only be answered given the previous context and
transcript details. The question should randomly start with How, Why, or What.

The output from this function is a llama_index class `EmbeddingQAFinetuneDataset`, which is a simple wrapper for a series of three dictionaries (`corpus`, `queries`, and `relevant_docs`).  The llama_index class is not absolutely necessary, but it is helpful in making transitions smoother when using the llama_index `SentenceTransformersFinetuneEngine` class for fine-tuning.  It takes roughly 80 seconds to generate 100 query/context pairs so a sample size of 300 takes about 4 minutes (much faster than if you were to do this manually!).

In [20]:
training_set = generator.generate_qa_embedding_pairs(train, qa_generation_prompt, 1)
val_set = generator.generate_qa_embedding_pairs(val, qa_generation_prompt, 1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.27it/s]


In [27]:
#EmbeddingQAFinetuneDataset has no len, so check length of queries instead
len(training_set.queries), len(val_set.queries)

(10, 5)

### Dataset Analysis

Always a good idea to check the quality of the pairs generated.  Most pairs will be high quality but some will not be, this is a chance for human intervention to adjust the questions manually to ensure the quality remains high. 

In [23]:
def show_qa_pairs(data: EmbeddingQAFinetuneDataset, print_results: bool=True):
    pairs = []
    for k, v in data.queries.items():
        doc_id = data.relevant_docs[k][0]
        context = data.corpus[doc_id]
        pairs.append((v, context))
    if print_results:
        for tup in pairs:
            print(f'Question: {tup[0]}\nContext: {tup[1]}\n\n')
    return pairs    

In [24]:
pairs = show_qa_pairs(val_set, print_results=True)

Question: How does Chase Jarvis describe his initial reaction to meditation and why did he feel that way?
Context: Tim Bilyeu Because I'm just taking it from a neurological perspective. Dr. Tim Jackson There you go. Tim Bilyeu Purely from a neurological perspective because I know what's happening. You're tapping into the parasympathetic nervous system, you're calming down the sympathetic. It's just, it's biochemistry, right? Dr. Tim Jackson If you believe in biochemistry, then give it a shot. I'm a huge advocate. I try not to sell it too hard because anytime someone's trying to sell you something, it feels inauthentic, but it's just given me a lot of joy. Dr. Tim Jackson Yeah. No, I think that it's like you ... Actually, you embraced it pretty quickly. Like Tim, I did not. I really felt that it was super soft, and I never thought of it as taking a step off my edge. It just felt like probably because my self-narrative ... Growing up, I was not good at sports. I did not feel overly tough

### Save to Disk  
Save to disk using your own filepaths, below is an example using the length of the sets as part of the filepath.

In [28]:
# training_set.save_json('./data/training_data_10.json')
# val_set.save_json('./data/validation_data_5.json')